In [ ]:
#!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### loading data to dataframes for first impression 

In [ ]:
filename = "DJIA_table_train.csv" 
mydir = "../data/train/"
DJ_df = pd.read_csv(mydir+filename, parse_dates=True)
mydir = "../data/interim/"

In [ ]:
DJ_df.head()

### exploring DJ_df 

In [ ]:
DJ_df.shape

In [ ]:
DJ_df.describe()

In [ ]:
sns.pairplot(DJ_df)

In [ ]:
DJ_df.any()

### missing values: some values are highly correlated so we can use this correlation to fill empty values

In [ ]:
# function for calculating linear regression and using it to fill values
def Fix_Empty(df1, df2):
    f1_index = df1.index[df1.apply(np.isfinite)].to_list()
    f2_index = df2.index[df2.apply(np.isfinite)].to_list()

    idx=list(set(f1_index) & set(f2_index))
    # since linearly correlated I can fill values easy using linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(df1[idx], df2[idx])
    f2_index = df2.index[df2.apply(np.isnan)].to_list()

    df2[f2_index] = intercept + slope * df1[f2_index]

### open and Adj_close

In [ ]:
Fix_Empty(DJ_df["Open"], DJ_df["Adj Close"])

In [ ]:
Fix_Empty(DJ_df["Adj Close"], DJ_df.Open)

### open and High

In [ ]:
Fix_Empty(DJ_df.Open, DJ_df.High)
Fix_Empty(DJ_df.High, DJ_df.Open)

### Open and Low

In [ ]:
Fix_Empty(DJ_df.Open, DJ_df.Low)
Fix_Empty(DJ_df.Low, DJ_df.Open)

### Open and Cose

In [ ]:
Fix_Empty(DJ_df.Open,DJ_df.Close)
Fix_Empty(DJ_df.Close,DJ_df.Open)

### Close and Low

In [ ]:
Fix_Empty(DJ_df.Low,DJ_df.Close)
Fix_Empty(DJ_df.Close,DJ_df.Low)

### Close and High

In [ ]:
Fix_Empty(DJ_df.High,DJ_df.Close)
Fix_Empty(DJ_df.Close,DJ_df.High)

### Close and Adj_close

In [ ]:
Fix_Empty(DJ_df.Close,DJ_df["Adj Close"])
Fix_Empty(DJ_df["Adj Close"],DJ_df.Close)

### Low and Adj_close

In [ ]:
Fix_Empty(DJ_df.Low,DJ_df["Adj Close"])
Fix_Empty(DJ_df["Adj Close"],DJ_df.Low)

### Low and High

In [ ]:
Fix_Empty(DJ_df.Low,DJ_df.High)
Fix_Empty(DJ_df.High,DJ_df.Low)

### See if I can fill Volume values.

#### First check if it depends on day of week

In [ ]:
DJ_df["Date"] = pd.to_datetime(DJ_df.Date)

In [ ]:
DJ_df["Day"] = DJ_df.Date.dt.dayofweek

In [ ]:
By_day_mean = DJ_df.groupby(by="Day").mean()
By_day_std = DJ_df.groupby(by="Day").std()
By_day_mean

In [ ]:
DJ_df.groupby(by="Day").agg(['mean', 'std'])

In [ ]:
plt.scatter(np.arange(5), By_day_mean.Volume)
plt.scatter(np.arange(5), By_day_std.Volume)

##### Check if depends on Day of Month

In [ ]:
DJ_df["DayOfMonth"] = DJ_df.Date.dt.day

In [ ]:
By_day_mean = DJ_df.groupby(by="DayOfMonth").mean()
By_day_std = DJ_df.groupby(by="DayOfMonth").std()

In [ ]:
By_day_mean

In [ ]:
plt.scatter(np.arange(1,32), By_day_mean.Volume)
plt.scatter(np.arange(1,32), By_day_std.Volume)

### check if Volume depends on Volume the day before

In [ ]:
pd.DataFrame([0, 1, 2]).shift(-1)

In [ ]:
laggedVol = DJ_df.Volume.shift(-1)
plt.scatter(laggedVol, DJ_df.Volume)

In [ ]:
laggedVol[0]

In [ ]:
DJ_df["Lag_Vol"] = DJ_df.Volume.shift(-1)

In [ ]:
DJ_df.Lag_Vol.loc[0] = DJ_df.Lag_Vol.loc[1]

In [ ]:
DJ_df.to_csv(mydir+"Clead_DJ")

In [ ]:
DJ_df = pd.read_csv(mydir+"Clead_DJ", parse_dates=True, index_col=0)

In [ ]:
#DJ_df.drop(columns="Unnamed: 0.1", inplace=True)
DJ_df.head()

In [ ]:
DJ_df.tail()

In [ ]:
train_idx2 = DJ_df.index[DJ_df.Lag_Vol.apply(np.isnan)].to_list()
len(train_idx2)

In [ ]:
a = DJ_df.Volume.shift(-2)
DJ_df.loc[train_idx2,'Lag_Vol'] = a[train_idx2]

In [ ]:
train_idx2 = DJ_df.index[DJ_df.Lag_Vol.apply(np.isnan)].to_list()
len(train_idx2)

In [ ]:
a = DJ_df.Volume.shift(-3)
DJ_df.loc[train_idx2,'Lag_Vol']=a[train_idx2]

In [ ]:
mask = DJ_df.loc[:, 'Lag_Vol'].isnull()]
#DJ_df.index[mask]

In [ ]:
sum(mask)

In [ ]:
train_idx2 = DJ_df.index[DJ_df.Lag_Vol.apply(np.isnan)].to_list()

In [ ]:
train_idx2 = DJ_df.index[DJ_df.Lag_Vol.apply(np.isnan)].to_list()
len(train_idx2)

In [ ]:
a = DJ_df.Volume.shift(-4)
DJ_df.loc[train_idx2,'Lag_Vol']=a[train_idx2]
train_idx2 = DJ_df.index[DJ_df.Lag_Vol.apply(np.isnan)].to_list()
train_idx2

In [ ]:
DJ_df.Lag_Vol[1583] = DJ_df.Lag_Vol[1582]

In [ ]:
DJ_df.to_csv(mydir+"Clean_DJ.csv")

In [ ]:
DJ_df.head()

In [ ]:
DJ_df.Lag_Vol.isnull().any()

### Random Forest for estimating missing Volume Values

In [ ]:
train_idx = DJ_df.index[DJ_df.Volume.apply(np.isfinite)].to_list()

test_idx = DJ_df.index[DJ_df.Volume.apply(np.isnan)].to_list()

In [ ]:
X_train = DJ_df.loc[train_idx,:]
y_train = X_train.Volume

X_train.drop(columns='Volume', inplace=True) 
X_train.drop(columns=["Date", "label"], inplace=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
X_test.shape

In [ ]:
X_train.reset_index(drop=True)
X_test.reset_index(drop=True)

y_train.reset_index(drop=True)
y_test.reset_index(drop=True)

In [ ]:
len(y_train)

In [ ]:
X_val = DJ_df.loc[test_idx,:]
X_val.drop(columns=["Date","label"],inplace=True)
y_val = X_val.Volume
X_val.drop(columns = 'Volume', inplace=True) 

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=3, random_state=42, n_estimators=100, verbose=1)
regr.fit(X_train, y_train)  

In [ ]:
from sklearn.metrics import r2_score

yhat = regr.predict(X_test)
r2_score(y_test, yhat)


In [ ]:
plt.scatter(yhat.T, y_test.T)

In [ ]:
yhat = regr.predict(X_train)
r2_score(y_train, yhat)

In [ ]:
meanY = np.mean(y_train)
yy = np.full_like(y_test, meanY)

In [ ]:
r2_score(y_test, yy)

In [ ]:
#print('random forest gives a r2_score on test of {:2.2f} % while a naive average gives r2_score of -0.002 so random forest it is!'.format(100 * r2_score(y_test, yhat)))

In [ ]:
yhat = regr.predict(X_val)

In [ ]:
DJ_df.loc[test_idx, "Volume"] = yhat

In [ ]:
#DJ_df.to_csv(mydir+"clean_DJIA.csv")

In [ ]:
DJ_df.isnull().any()

In [ ]:
DJ_df.head()

In [ ]:
#DJ_df.to_csv(mydir+"Clead_DJ")

In [ ]:
a = DJ_df["Adj Close"] - DJ_df["Adj Close"].shift(1)
a[a>=0] = 1
a[a<0] = 0
# a[1583]=0

In [ ]:
a[0] = 0

In [ ]:
DJ_df.label = a

In [ ]:
DJ_df.head()

In [ ]:
#DJ_df.to_csv(mydir+"clean_DJIA.csv")

### Dimentionality reduction using PCA 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(DJ_df.loc[:,["Open","High","Low","Close","Volume","Adj Close","Lag_Vol"]])

In [ ]:
print(pca.explained_variance_ratio_)  

In [ ]:
print(pca.singular_values_) 

In [ ]:
b = pca.transform(DJ_df.loc[:, ["Open","High","Low","Close","Volume","Adj Close","Lag_Vol"]])

In [ ]:
b.shape

In [ ]:
DJ_df["1st_PC"] = b[:,0]
DJ_df["2nd_PC"] = b[:,1]

In [ ]:
DJ_df.to_csv(mydir+"clean_DJIA.csv")

In [ ]:
DJ_df.head()

In [ ]:
#DJ_df.drop(columns=["Open","High","Low","Close","Volume","Adj Close","Lag_Vol"],inplace=True)